In [1]:
import torch
torch.cuda.is_available()

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [7]:
!python main.py --model_dir experiments/base_resnet18

2022-11-06 21:56:55.620489: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-06 21:56:55.833469: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-06 21:56:56.530567: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-11-06 21:56:56.530663: W tensorflow/stream_executor

In [4]:
import utils
import os, logging

model_dir = 'experiments/base_resnet18'
log_path = os.path.join(model_dir, 'train.log')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

if logger.handlers:
    # Logging to a file
    file_handler = logging.FileHandler(log_path)
    file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
    logger.addHandler(file_handler)

# Create the input data pipeline
logging.info("Loading the datasets...")

INFO:root:Loading the datasets...


In [6]:
import utils
# Set the logger
utils.set_logger(log_path)

# Create the input data pipeline
logging.info("Loading the datasets...")

INFO:root:Loading the datasets...


In [3]:
os.path.join(model_dir, 'train.log')

'experiments/base_resnet18/train.log'

In [ ]:
# # Extract data from log files and plot

filename = '/content/drive/MyDrive/CS762/experiments/base_resnet18/train.log'

train_acc = []
loss = []
eval_acc = []

with open(filename) as file:
  for line in file:
    l = line.rstrip()
    if 'Train metrics' in l:
      train_acc.append(float(l[-19:-14]))
      loss.append(float(l[-5:]))
    if 'Eval metrics' in l:
      eval_acc.append(float(l[-19:-14]))

import matplotlib.pyplot as plt
plt.plot(train_acc)
plt.plot(eval_acc)
plt.show()


In [ ]:
#drive.flush_and_unmount()